For Week 4 I have to answer to questions below

1) A description of the problem and a discussion of the background. (15 marks)

2) A description of the data and how it will be used to solve the problem. (15 marks)

Okay let's start with the first question

1) A Description of the problem and a discussion of the background

"Where is the dangerous areas of Baltimore, MD, USA?" I am moving to baltimore in a near future and wants visualize to see which area is dangerous and which one is not.

2) A description of the data and how it will be used to solve the problem

Baltimore Zipcode: http://ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=md&StName=Georgia&StFIPS=13&FIPS=24510

Baltimore Crime Data : https://data.baltimorecity.gov/datasets/part1-crime-data/explore?location=39.300436%2C-76.606438%2C11.36&showTable=true